In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import csv

from nc_processing import *
from analysis import * 

%matplotlib inline

In [2]:
# define the data directory to search for files using lists
ceda_dir='/badc/cmip6/data/CMIP6/{project}/{centre}/{model}/{exp}/{run}/{domain}/{var}/gn/latest/'
# define the base of the filename for the output file(s) using lists, note this matches the file format in the data directory except for the date_4_file section and the missing .nc. 
out_base='{var}_{domain}_{model}_{exp}_{run}_gn_{time_range}' # all that's missing is a .nc but we can add that later.

In [5]:
def global_mean(var,exp,dates,project):
    data_dir='/home/users/quigley/Projects/geoengineering_summer' #Changed from Pete's directory
    model='UKESM1-0-LL'
    centre='MOHC'
    
    if var == "evspsblpot":
        domain = "Emon"  # source documentation said it used Emon, not Amon, so I gave this a try. Didn't work
    else:
        domain='Amon' # pr is an Amon variable.
    
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']
    grid='gn'
    season='ANN'
    time_files=1
    
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')

    # load the arguments into a list then... 
    args=[season,dates,data_dir,model,centre,var,domain,exp,project,runs,grid,time_files]
    ds_mean, ds_std = get_ens_seasonal_mean_std(*args) # ... unpack them into the function.
    
    # multiply each temperature value with its corresponding fraction of total area, then sum to find the global mean.
    return (ds_mean[var]*da_weight).sum() - 273.15 # convert from K to C.


In [6]:
def moderated_exacerbated(var):
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 
    # NOTE - to do this for another model, you'll need to look up the run number used in the piControl experiment which stores the fx variables.
    # search on CEDA archive for "areacella" and "<MODEL NAME>" and it should show you.

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')
    # land area = gricell area * land fraction ([0-100] * 0.01)
    da_land_area = ds_area['areacella'] * ds_land['sftlf'] * 0.01 
    # Now let's create a land area weighting
    da_land_weight = da_land_area / da_land_area.sum()
    
    data_dir='/home/users/quigley/Projects/geoengineering_summer'

    # Set variables that are common to all experiments.
    model='UKESM1-0-LL'
    centre='MOHC' 
    
    if var == "evspsblpot":
        domain = "Emon"
    else:
        domain='Amon' # pr is an Amon variable.
    grid='gn'
    season='ANN'

    # specify a list of runs
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']

    # Load G6sulfur ensemble-mean datasets
    exp='G6sulfur'
    project='GeoMIP'
    dates=['2070-01-01','2100-01-01']
    ds_G6sulfur_mean, ds_G6sulfur_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='ScenarioMIP'
    exp='ssp585'
    dates=['2070-01-01','2100-01-01']
    ds_ssp585_mean, ds_ssp585_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='CMIP'
    exp='historical'
    dates=['1960-01-01','1990-01-01']
    ds_historical_mean, ds_historical_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.
    
    # Call my better_worse_off function, passing in the data_arrays for the variable of interest rather than the full datasets.
    better, worse, dunno = better_worse_off(ds_G6sulfur_mean[var], ds_G6sulfur_std[var], ds_ssp585_mean[var], ds_ssp585_std[var], ds_historical_mean[var], ds_historical_std[var], 90, 0.05)
    
    #Calculating the moderated/exacerbated arrays before determining significance
    global_moderated = 100.*((better + 0) * da_weight).values
    global_exacerbated = 100.*((worse + 0) * da_weight).values
    land_moderated = 100.*((better + 0) * da_land_weight).values
    land_exacerbated = 100.*((worse + 0) * da_land_weight).values
    
    return global_moderated, global_exacerbated, land_moderated, land_exacerbated

In [7]:
global_moderated, global_exacerbated, land_moderated, land_exacerbated = moderated_exacerbated("pr")

loading existing files pr_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files pr_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files pr_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN


In [8]:
global_moderated

array([[6.19718435e-05, 6.19718435e-05, 6.19718435e-05, ...,
        6.19718435e-05, 6.19718435e-05, 6.19718435e-05],
       [1.85886529e-04, 1.85886529e-04, 1.85886529e-04, ...,
        1.85886529e-04, 1.85886529e-04, 1.85886529e-04],
       [3.09712755e-04, 3.09712755e-04, 3.09712755e-04, ...,
        3.09712755e-04, 3.09712755e-04, 3.09712755e-04],
       ...,
       [3.09712755e-04, 3.09712755e-04, 3.09712755e-04, ...,
        3.09712755e-04, 3.09712755e-04, 3.09712755e-04],
       [1.85886529e-04, 1.85886529e-04, 1.85886529e-04, ...,
        1.85886529e-04, 1.85886529e-04, 1.85886529e-04],
       [6.19718435e-05, 6.19718435e-05, 6.19718435e-05, ...,
        6.19718435e-05, 6.19718435e-05, 6.19718435e-05]])

In [9]:
def stats(var,exp,dates,project):
    data_dir='/home/users/quigley/Projects/geoengineering_summer' #Changed from Pete's directory
    model='UKESM1-0-LL'
    centre='MOHC'
    
    if var == "evspsblpot":
        domain = "Emon"  # source documentation said it used Emon, not Amon, so I gave this a try. Didn't work
    else:
        domain='Amon' # pr is an Amon variable.
    
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']
    grid='gn'
    season='ANN'
    time_files=1
    
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')

    # load the arguments into a list then... 
    args=[season,dates,data_dir,model,centre,var,domain,exp,project,runs,grid,time_files]
    ds_mean, ds_std = get_ens_seasonal_mean_std(*args) # ... unpack them into the function.
    
    return ds_mean, ds_std


In [10]:
var = "tas"
exp = "G6sulfur"
exp2 = "G6solar"
dates = ['2070-01-01','2100-01-01']
project = "GeoMIP"

ds_mean, ds_std = stats(var, exp, dates, project)

ds_mean_2, ds_std_2 = stats(var, exp2, dates, project)

num_years = 90 # the t-test needs to know how long our sample is.

# ttest_sub returns a numpy array of P-values, where P is between 0 and 1. for 95% significance P is below 0.05
ttest_pvalue = ttest_sub(ds_mean[var],ds_std[var],num_years,ds_mean_2[var],ds_std_2[var],num_years)

# Let's put the ttest results into the same format as our xarray datasets
ds_ttest = xr.full_like(ds_mean, 0.0) # copy dataset format from ds_mean and set data values to 0.
ds_ttest.rename(name_dict={var:'p_value'}) # rename the variable to p_value
ds_ttest['p_value'] = (['lat','lon'],ttest_pvalue<0.05) # Fill in the blank values with our ttest results

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN


In [11]:
ttest_pvalue

array([[0.00044794, 0.00044436, 0.00044608, ..., 0.00044305, 0.00043472,
        0.00044657],
       [0.0006349 , 0.00063315, 0.00064695, ..., 0.0005913 , 0.00060991,
        0.00061167],
       [0.00133728, 0.00142585, 0.00140628, ..., 0.0012091 , 0.00127   ,
        0.00129372],
       ...,
       [0.16412996, 0.16855228, 0.16743845, ..., 0.14194397, 0.14966384,
        0.15866436],
       [0.2001084 , 0.20268068, 0.20492612, ..., 0.18832965, 0.19431596,
        0.19735214],
       [0.17840334, 0.17876235, 0.1792337 , ..., 0.17603959, 0.17656434,
        0.17723068]])

In [12]:
def ttest(var,exp,dates,project):
    ds_mean, ds_std = stats(var, exp, dates, project)
    ds_mean_2, ds_std_2 = stats(var, "historical", ['1960-01-01','1990-01-01'], "CMIP")

    num_years = 90 # the t-test needs to know how long our sample is.

    # ttest_sub returns a numpy array of P-values, where P is between 0 and 1. for 95% significance P is below 0.05
    ttest_pvalue = ttest_sub(ds_mean[var],ds_std[var],num_years,ds_mean_2[var],ds_std_2[var],num_years)

    return ttest_pvalue

In [13]:
def moderated_exacerbated_with_significance(moderated,exacerbated,ttest_pvalue):

    sig_mod = 0
    insig_mod = 0
    sig_exac = 0
    insig_exac = 0
    dunno = 0

    for i in range(0,len(ttest_pvalue[:,0])-1): #longitude
        for j in range(0,len(ttest_pvalue[0,:])-1): #latitude
            #print(ttest_pvalue[i,j])
            if moderated[i,j]-exacerbated[i,j] > 0 and ttest_pvalue[i,j] < 0.05:
                sig_mod = sig_mod + moderated[i,j]
                insig_exac = insig_exac + exacerbated[i,j]
            elif moderated[i,j]-exacerbated[i,j] > 0 and ttest_pvalue[i,j] >= 0.05:
                insig_mod = insig_mod + moderated[i,j]
                insig_exac = insig_exac + exacerbated[i,j]
            elif moderated[i,j]-exacerbated[i,j] < 0 and ttest_pvalue[i,j] < 0.05:
                sig_exac = sig_exac + exacerbated[i,j]
                insig_mod = insig_mod + moderated[i,j]
            elif moderated[i,j]-exacerbated[i,j] < 0 and ttest_pvalue[i,j] <= 0.05:
                insig_exac = insig_exac + exacerbated[i,j]
                insig_mod = insig_mod + moderated[i,j]

    return sig_mod, insig_mod, sig_exac, insig_exac

In [ ]:
#Compare where climate change is significant vs insignificant
#pandas

In [14]:
def calculate(var, metric):
    control_mean=global_mean(var,'historical',['1960-01-01','1990-01-01'],'CMIP').values #1960-1990 historical data
    geo_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP').values  
    solar_mean=global_mean(var,'G6solar',['2070-01-01','2100-01-01'],'GeoMIP').values
    sulfur_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP').values
    warming_mean=global_mean(var,"ssp585",['2070-01-01','2100-01-01'],"ScenarioMIP").values
    ds_mean, ds_std = stats(var,'G6solar',['2070-01-01','2100-01-01'],'GeoMIP')
    ds_mean_2, ds_std_2 = stats(var, "historical", ['1960-01-01','1990-01-01'], "CMIP")
    num_years = 90 # the t-test needs to know how long our sample is.
    # ttest_sub returns a numpy array of P-values, where P is between 0 and 1. for 95% significance P is below 0.05
    ttest_pvalue = ttest_sub(ds_mean[var],ds_std[var],num_years,ds_mean_2[var],ds_std_2[var],num_years) #calculates p value for each value in array
    global_moderated, global_exacerbated, land_moderated, land_exacerbated = moderated_exacerbated(var)
    
    geo_anom = geo_mean - control_mean
    sulfur_solar_anom = sulfur_mean - solar_mean
    geo_warming_anom = geo_mean - warming_mean
    warming_anom = warming_mean - control_mean

    if metric == "global_mean_anomaly_control":
        return geo_mean - control_mean
    elif metric == "global_mean_anomaly_solar":
        return sulfur_mean - solar_mean
    elif metric == "global_mean_anomaly_warming":
        return geo_mean - warming_mean
    elif metric == "moderated_exacerbated":
        if geo_anom < warming_anom:
            return "moderated"
        elif warming_anom < geo_anom:
            return "exacerbated"
        else:
            return "N/A"
    elif metric == "global_sig_mod":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(global_moderated,global_exacerbated,ttest_pvalue)
        return sig_mod
    elif metric == "global_insig_mod":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(global_moderated,global_exacerbated,ttest_pvalue)
        return insig_mod
    elif metric == "global_sig_exac":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(global_moderated,global_exacerbated,ttest_pvalue)
        return sig_exac
    elif metric == "global_insig_exac":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(global_moderated,global_exacerbated,ttest_pvalue)
        return insig_exac
    elif metric == "land_sig_mod":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(land_moderated,land_exacerbated,ttest_pvalue)
        return sig_mod
    elif metric == "land_insig_mod":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(land_moderated,land_exacerbated,ttest_pvalue)
        return insig_mod
    elif metric == "land_sig_exac":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(land_moderated,land_exacerbated,ttest_pvalue)
        return sig_exac
    elif metric == "land_insig_exac":
        sig_mod, insig_mod, sig_exac, insig_exac = moderated_exacerbated_with_significance(land_moderated,land_exacerbated,ttest_pvalue)
        return insig_exac
    else:
        return 0

In [19]:
calculate("tas", "global_sig_exac")

loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN


0

In [20]:
calculate("tas", "global_sig_mod")

loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN


99.46733453360821

In [21]:
calculate("tas", "global_insig_exac")

loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN


0.0

In [22]:
calculate("tas", "global_insig_mod")

loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN


0